# this script builds a loop over all documents to extract the beccessary information

In [8]:
#load all neccessary packages
import pandas
import os
import requests
import re
from bs4 import BeautifulSoup

In [10]:
#define the directory of the datafiles I want to loop over
user = '/Users/natalies_macbook/Documents/' #change this to your folder where GitHub sits
directory = user + 'GitHub/INSA/EU/relevant/'

In [12]:
#define all patterns

#search for UN to add the UN dummy indicator
UN_pattern = re.search(r'UN',data)

UN = None
for i in data:
    if UN_pattern == None:
        UN = 0
    else:
        UN = 1
#get the document base Information       
url= re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data) #finds the url through the href tag
#publication_date = re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
#title = re.findall(r'class="oj-doc-ti"[^>]*>([\s\S]*?)<\/p>', data)
title = soup.find('p', class_='oj-doc-ti').get_text(strip=True)
celex= re.findall(r'uri=CELEX%([^"]+)', data) # get the celex id
publication_date = soup.find('p', class_='oj-hd-date').get_text(strip=True)

#get the document information
decision_match = re.findall(r'(COUNCIL.*?)</p', data)
legal_decision = re.findall(r'<p[^>]*>(.*?(amending|implementing).*?)</p>',data)
legal_base = re.findall(r'<p[^>]*>(.*?regard to.*?)</p>',data)

#check for list updates, removals or deletions
action_intended = re.findall(r'<p class="(oj-normal|oj-bold)">\s*(.*?\s*(?:removed|updated|deleted|replaced)\s*.*?)\s*</p>',data)
action_intended

NameError: name 'data' is not defined

In [14]:
listing_position = re.findall(r'class="oj-normal">(\d+.*?)',data)
print(listing_position)
name_2 = re.findall(r'<p class="oj-normal">\s*<span class="oj-bold">(.*?)</span>', data)
print(name_2)
un_sanction_date = re.findall(r'class="oj-normal">Date of UN designation(.*?)</p>',data,re.DOTALL)
print(date_pattern)
identifying_information =re.findall(r'>(.*?Date of Birth:.*?)</p>',data)
print(identifying_information)

NameError: name 'data' is not defined

In [16]:
#court decision matching
name_pattern_court = r'Orders (.+?) to pay'
#for dismissal entity name
# Create patterns
delist_pattern = r'Annuls'
delisting_case_pattern = r'Annuls (.+?);2.Orders'
name_pattern2 = r'in so far as they concern (.+?);2.Orders'
name_pattern1 = r'Orders (.+?) to pay'

# Create a list to store all court decisions
court_decisions = []

# Iterate over the content
for i in content:
    # Get the content of the court case
    name = re.findall(r'<p class="oj-normal">(.*?)</p>', i)
    text = ''.join(name)
    
    # Initialize variables
    Delisting = None
    delisting_case = None
    entity_name = None
    
    # Check for delisting
    if re.search(delist_pattern, text):
        Delisting = 1
    else:
        Delisting = 0
    if Delisting == 1:
        match = re.search(name_pattern2, text)
        if match:
            entity_name = match.group(1)
    if Delisting == 0:
        match = re.search(name_pattern1, text)
        if match:
            entity_name = match.group(1)
    
    # Extract delisting case
    match = re.search(delisting_case_pattern, text)
    if match:
        delisting_case = match.group(1)

    # Create dictionary for current court decision
    court_decision = {
        'entity_name': entity_name,
        'Delisting': Delisting,
        'Delisting_case': delisting_case,
        'url': re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data),
        'celex' : re.findall(r'uri=CELEX%([^"]+)', data),
        'publication_date': re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data),
        'date': re.findall(r'<p class="oj-doc-ti">of (\d+ \w+ \d+)</p>', data),
        'title': re.findall(r'<p class="oj-doc-ti" id="[^"]*">(.*?)</p>', data),
        'law_number': re.findall(r'<p class="oj-hd-oj">C (\d+/\d+)</p>', data),
        'case_number': re.findall(r'Case\sT-\d+/\d+', data),
        'legal_base': re.findall(r'\((CFSP|EU)\)\s*(\d+/\d+)', data)
    }
    
    # Append the current decision dictionary to the list
    court_decisions.append(court_decision)

# Print the list of dictionaries
print(court_decisions, encoding = 'UTF-8') #encoding ändern oder string subsitute (string punkt replace xa0 mit .)
#now extract th information
# Initialize variables
Delisting = None
delisting_case = None
entity_name = None
#create patterns
delist_pattern = r'The Court: 1. Annuls'
delisting_case_pattern = r'Annuls (.+?);2.Orders'
name_pattern = r'in so far as they concern (.+?);2.Orders'
#create empty dic
results = {}

for i in content:
    #get the content of the court case
    name = re.findall(r'<p class="oj-normal">(.*?)</p>', i)
    text = ''.join(name)
    if re.search(delist_pattern, text):
        Delisting = 0
        print('Dismissal')
    else:
        Delisting = 1  # Assuming 1 for non-dismissal cases
    match = re.search(delisting_case_pattern, text)
    if match:
        delisting_case = match.group(1)
    # Extract entity name
    match = re.search(name_pattern, text)
    if match:
        entity_name = match.group(1)
#now get it into a dictionary format
court_decision = {}
court_decision['entity_name'] = entity_name
court_decision['Delisting'] = Delisting
court_decision['Delisting_case']= delisting_case
court_decision['url'] = re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data)
court_decision['publication_date'] =re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
court_decision['date'] =  re.findall(r'<p class="oj-doc-ti">of (\d+ \w+ \d+)</p>', data)
court_decision['title']= re.findall(r'<p class="oj-doc-ti" id="[^"]*">(.*?)</p>', data)
court_decision['law_number'] = re.findall(r'<p class="oj-hd-oj">C (\d+/\d+)',data)
court_decision['case_number'] = re.findall(r'Case\sT-\d+/\d+', data)
court_decision['legal_base'] = re.findall(r'\((CFSP|EU)\)\s*(\d+/\d+)', data)


NameError: name 'content' is not defined

In [ ]:
#define all the patterns
url = re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data) #finds the url through the href tag
#publication_date = re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
#title = re.findall(r'class="oj-doc-ti"[^>]*>([\s\S]*?)<\/p>', data)
title = soup.find('p', class_='oj-doc-ti').get_text(strip=True)
celex= re.findall(r'uri=CELEX%([^"]+)', data) # get the celex id
publication_date = soup.find('p', class_='oj-hd-date').get_text(strip=True)

#get the document information
decision_match = re.findall(r'(COUNCIL.*?)</p', data)
legal_decision = re.findall(r'<p[^>]*>(.*?(amending|implementing).*?)</p>',data)
legal_base = re.findall(r'<p[^>]*>(.*?regard to.*?)</p>',data)

#check for list updates, removals or deletions
action_intended = re.findall(r'<p class="(oj-normal|oj-bold)">\s*(.*?\s*(?:removed|updated|deleted|replaced)\s*.*?)\s*</p>',data)
action_intended        

In [28]:

# Write the loop
for dirpath, dirnames, filenames in os.walk(directory):
    for filename in filenames:
        if filename.endswith('.txt'):
            file_path = os.path.join(dirpath, filename)
            try:
                with open(file_path, encoding='utf-8') as f:
                    data = f.read()
            except UnicodeDecodeError:
                with open(file_path, encoding='latin-1') as f:
                    data = f.read()
                    
            soup = BeautifulSoup(data, 'html.parser')
            
            # Define all the patterns
            url = re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data) # finds the url through the href tag
            celex = re.findall(r'uri=CELEX%([^"]+)', data) # get the celex id
            title = soup.find('p', class_='oj-doc-ti').get_text(strip=True) if soup.find('p', class_='oj-doc-ti') else ''
            publication_date = soup.find('p', class_='oj-hd-date').get_text(strip=True) if soup.find('p', class_='oj-hd-date') else ''
            decision_match = re.findall(r'(COUNCIL.*?)</p', data)
            legal_decision = re.findall(r'<p[^>]*>(.*?(amending|implementing).*?)</p>', data)
            legal_base = re.findall(r'<p[^>]*>(.*?regard to.*?)</p>', data)
            action_intended = re.findall(r'<p class="(oj-normal|oj-bold)">\s*(.*?\s*(?:removed|updated|deleted|replaced)\s*.*?)\s*</p>', data)
            
            rows = soup.find_all('tr', class_='oj-table')  # Finds all elements tr = tablerow of the class oj-table
            for row in rows[1:]:  # Skip the header row
                cells = row.find_all('td')  # Finds all td table data in one tablerow
                if len(cells) < 5:
                    continue
                name = cells[1].get_text(strip=True)
                identifying_info = cells[2].get_text(strip=False)  # Extracting the identifying information but keeping the newline 
                reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p'))
                date_of_listing = cells[4].get_text(strip=True)
                
                # Since url and celex are lists, extract the first element if they exist
                url_str = url[0] if url else ''
                celex_str = celex[0] if celex else ''
                
                sanction_dict = {
                    'URL': url_str,
                    'CELEX': celex_str,
                    'law_number': title,
                    'Publication Date': publication_date,
                    'legal_base': legal_base[0] if legal_base else '',
                    'Name': name,
                    'Identifying Information': identifying_info,
                    'Reason': reasons,
                    'Date of Listing': date_of_listing,
                    'Decision Match': decision_match[0] if decision_match else '',
                    'Legal Decision': legal_decision[0][0] if legal_decision else '',
                    'Action Intended': action_intended[0][1] if action_intended else ''
                }
                sanction_list.append(sanction_dict)

# At this point, sanction_list contains the parsed data
# You can print or process the sanction_list as needed
print(sanction_list)


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [20]:
# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# Find all rows in the table
rows = soup.find_all('tr', class_='oj-table')

# List to store the parsed data
sanction_list = []

# Iterate through the rows and extract the necessary information
for row in rows[1:]:  # Skip the header row
    cells = row.find_all('td')
    if len(cells) < 5:
        continue

    name = cells[1].get_text(strip=True)
    identifying_info = cells[2].get_text(strip=True)  # Extracting the identifying information
    reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p'))
    date_of_listing = cells[4].get_text(strip=True)
    
    sanction_list.append({
        'URL': url,
        'CELEX':celex,
        'law_number' :decision_match[0],
        'Publication Date': publication_date,
        'legal_base' : legal_decision[0],
        'Name': name,
        'Identifying Information': identifying_info,
        'Reason': reasons,
        'Date of Listing': date_of_listing
    })

# Display the results
for person in sanction_list:
    print(person)

NameError: name 'data' is not defined